In [83]:
import sys
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [84]:
sys.path.append(os.path.abspath('../source'))

from BD_connection import get_sqlalchemy_engine

conexion = get_sqlalchemy_engine()

Conexión a la base de datos exitosa.


In [85]:
df = pd.read_sql_query("SELECT * FROM raw_grammy", conexion)
print(df.head())

   year                              title               published_at  \
0  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
1  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
2  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
3  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   
4  2019  62nd Annual GRAMMY Awards  (2019)  2020-05-19T05:10:28-07:00   

                  updated_at            category     nominee         artist  \
0  2020-05-19T05:10:28-07:00  Record Of The Year     Bad Guy  Billie Eilish   
1  2020-05-19T05:10:28-07:00  Record Of The Year     Hey, Ma       Bon Iver   
2  2020-05-19T05:10:28-07:00  Record Of The Year     7 rings  Ariana Grande   
3  2020-05-19T05:10:28-07:00  Record Of The Year  Hard Place         H.E.R.   
4  2020-05-19T05:10:28-07:00  Record Of The Year        Talk         Khalid   

                                             workers  \
0  Finneas O'Connell, producer

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4810 entries, 0 to 4809
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          4810 non-null   int64 
 1   title         4810 non-null   object
 2   published_at  4810 non-null   object
 3   updated_at    4810 non-null   object
 4   category      4810 non-null   object
 5   nominee       4804 non-null   object
 6   artist        2970 non-null   object
 7   workers       2620 non-null   object
 8   img           3443 non-null   object
 9   winner        4810 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 375.9+ KB


In [87]:
null_values= df.isnull().sum()
print("Valores nulos:\n", null_values)

Valores nulos:
 year               0
title              0
published_at       0
updated_at         0
category           0
nominee            6
artist          1840
workers         2190
img             1367
winner             0
dtype: int64


In [88]:
df["winner"].value_counts()

winner
1    4810
Name: count, dtype: int64

In [89]:
duplicate_values= df[df.duplicated(keep=False)]
print("Datos duplicados:\n", duplicate_values)

Datos duplicados:
 Empty DataFrame
Columns: [year, title, published_at, updated_at, category, nominee, artist, workers, img, winner]
Index: []


In [90]:
df.loc[df["nominee"].isna()]

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
2261,2000,43rd Annual GRAMMY Awards (2000),2017-11-28T00:03:45-08:00,2019-09-10T01:11:09-07:00,"Remixer of the Year, Non-Classical",None,None,None,None,1
2359,1999,42nd Annual GRAMMY Awards (1999),2017-11-28T00:03:45-08:00,2019-09-10T01:09:02-07:00,"Remixer Of The Year, Non-Classical",None,None,None,None,1
2454,1998,41st Annual GRAMMY Awards (1998),2017-11-28T00:03:45-08:00,2019-09-10T01:08:19-07:00,"Remixer Of The Year, Non-classical",None,None,None,None,1
2547,1997,40th Annual GRAMMY Awards (1997),2017-11-28T00:03:45-08:00,2019-09-10T01:07:37-07:00,"Remixer Of The Year, Non-Classical",None,None,None,None,1
4525,1965,8th Annual GRAMMY Awards (1965),2017-11-28T00:03:45-08:00,2019-09-10T01:06:59-07:00,Best New Country & Western Artist,None,None,None,None,1
4573,1964,7th Annual GRAMMY Awards (1964),2017-11-28T00:03:45-08:00,2019-09-10T01:06:11-07:00,Best New Country & Western Artist Of 1964,None,None,None,None,1


In [91]:
df = df.dropna(subset=["nominee"])
df.loc[df["nominee"].isna()]

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner


In [92]:
df.loc[df["artist"].isna()]

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
16,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Song Of The Year,Bad Guy,None,"Billie Eilish O'Connell & Finneas O'Connell, s...",https://www.grammy.com/sites/com/files/styles/...,1
17,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Song Of The Year,Always Remember Us This Way,None,"Natalie Hemby, Lady Gaga, Hillary Lindsey & Lo...",https://www.grammy.com/sites/com/files/styles/...,1
18,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Song Of The Year,Bring My Flowers Now,None,"Brandi Carlile, Phil Hanseroth, Tim Hanseroth ...",https://www.grammy.com/sites/com/files/styles/...,1
19,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Song Of The Year,Hard Place,None,"Ruby Amanfu, Sam Ashworth, D. Arcelious Harris...",https://www.grammy.com/sites/com/files/styles/...,1
20,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Song Of The Year,Lover,None,"Taylor Swift, songwriter (Taylor Swift)",https://www.grammy.com/sites/com/files/styles/...,1
...,...,...,...,...,...,...,...,...,...,...
4805,1958,1st Annual GRAMMY Awards (1958),2017-11-28T00:03:45-08:00,2019-09-10T01:11:09-07:00,Best Classical Performance - Instrumentalist (...,Tchaikovsky: Piano Concerto No. 1 In B Flat Mi...,None,"Van Cliburn, artist (Symphony Of The Air Orche...",None,1
4806,1958,1st Annual GRAMMY Awards (1958),2017-11-28T00:03:45-08:00,2019-09-10T01:11:09-07:00,Best Classical Performance - Instrumentalist (...,Segovia Golden Jubilee,None,"Andres Segovia, artist",https://www.grammy.com/sites/com/files/styles/...,1
4807,1958,1st Annual GRAMMY Awards (1958),2017-11-28T00:03:45-08:00,2019-09-10T01:11:09-07:00,Best Classical Performance - Chamber Music (In...,Beethoven: Quartet 130,None,"Hollywood String Quartet (Alvin Dinkin, Paul S...",None,1
4808,1958,1st Annual GRAMMY Awards (1958),2017-11-28T00:03:45-08:00,2019-09-10T01:11:09-07:00,Best Classical Performance - Vocal Soloist (Wi...,Operatic Recital,None,None,None,1


In [93]:
artist_workers_null = df[df['artist'].isna() & df['workers'].isna()]
artist_workers_null

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
24,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Billie Eilish,None,None,None,1
25,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Black Pumas,None,None,None,1
26,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Lil Nas X,None,None,None,1
27,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Lizzo,None,None,None,1
28,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best New Artist,Maggie Rogers,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...
4726,1960,3rd Annual GRAMMY Awards (1960),2017-11-28T00:03:45-08:00,2019-09-10T01:07:37-07:00,Best Classical Performance - Vocal Soloist,A Program Of Song - Leontyne Price Recital,None,None,None,1
4745,1960,3rd Annual GRAMMY Awards (1960),2017-11-28T00:03:45-08:00,2019-09-10T01:07:37-07:00,Best New Artist Of 1960,Bob Newhart,None,None,None,1
4762,1959,2nd Annual GRAMMY Awards (1959),2017-11-28T00:03:45-08:00,2019-09-10T01:06:59-07:00,Best Classical Performance - Vocal Soloist (Wi...,Bjoerling In Opera,None,None,None,1
4781,1959,2nd Annual GRAMMY Awards (1959),2017-11-28T00:03:45-08:00,2019-09-10T01:06:59-07:00,Best New Artist Of 1959,Bobby Darin,None,None,None,1


In [94]:
category_counts = artist_workers_null['category'].value_counts()
print(category_counts)

category
Best New Artist                                                                              50
Producer Of The Year, Non-Classical                                                          22
Producer Of The Year, Classical                                                              22
Classical Producer Of The Year                                                               18
Producer Of The Year (Non-Classical)                                                         10
Producer Of The Year                                                                         10
Best New Artist Of The Year                                                                   9
Best Classical Vocal Soloist Performance                                                      7
Best Classical Vocal Performance                                                              4
Best Small Ensemble Performance (With Or Without Conductor)                                   4
Best Classical Performance - In

In [95]:
for category, group in artist_workers_null.groupby('category'):
    print(f"\nCategoría: {category}")
    print(f"Frecuencia: {len(group)}")
    # Mostrar hasta 5 ejemplos de 'nominee' (o todos si son menos)
    nominees = group['nominee'].dropna().unique()[:5]
    print("Ejemplos de 'nominee':")
    for nominee in nominees:
        print(f"  - {nominee}")


Categoría: Best Classical Performance - Instrumental Soloist Or Soloists (With Or Without Orchestra)
Frecuencia: 4
Ejemplos de 'nominee':
  - Horowitz - The Studio Recordings, New York 1985
  - Switched-On-Bach
  - Horowitz In Concert (Haydn, Schumann, Scriabin, Debussy, Mozart, Chopin)
  - Baroque Guitar (Works Of Bach, Sanz, Weiss, Etc.)

Categoría: Best Classical Performance - Vocal Soloist
Frecuencia: 1
Ejemplos de 'nominee':
  - A Program Of Song - Leontyne Price Recital

Categoría: Best Classical Performance - Vocal Soloist (With Or Without Orchestra)
Frecuencia: 2
Ejemplos de 'nominee':
  - Bjoerling In Opera
  - Operatic Recital

Categoría: Best Classical Vocal Performance
Frecuencia: 4
Ejemplos de 'nominee':
  - An Italian Songbook - Works Of Bellini, Donizetti, Rossini
  - The Impatient Lover - Italian Songs By Beethoven, Schubert, Mozart
  - The Art Of Arleen Auger (Works Of Larsen, Purcell, Schumann, Mozart)
  - Kathleen Battle At Carnegie Hall (Handel, Mozart, Liszt, Stra

In [96]:
mask_both_null = df['artist'].isna() & df['workers'].isna()
subset = df[mask_both_null].copy()

problematic_categories = [
    'Best Small Ensemble Performance (With Or Without Conductor)',
    'Most Promising New Classical Recording Artist',
    'Best Classical Performance - Instrumental Soloist Or Soloists (With Or Without Orchestra)',
    'Best Performance - Instrumental Soloist Or Soloists (With Or Without Orchestra)',
    'Best Classical Vocal Soloist',  # Ejemplo: "The Girl With Orange Lips"
    'Best New Classical Artist',
    'Best Classical Vocal Performance',
    'Best Classical Vocal Soloist Performance' 
]

mask_problematic = subset['category'].isin(problematic_categories)
if mask_problematic.any():
    print(f"\nEliminando {mask_problematic.sum()} filas de categorías problemáticas:")
    print(subset[mask_problematic][['category', 'nominee']].head())
    df = df[~df.index.isin(subset[mask_problematic].index)]



Eliminando 25 filas de categorías problemáticas:
                                               category  \
2369  Best Small Ensemble Performance (With Or Witho...   
2464  Best Small Ensemble Performance (With Or Witho...   
2557  Best Small Ensemble Performance (With Or Witho...   
2558                   Best Classical Vocal Performance   
2646  Best Small Ensemble Performance (With Or Witho...   

                                                nominee  
2369  Colors Of Love - Works Of Thomas, Stucky, Tave...  
2464                      Reich: Music For 18 Musicians  
2557  Hindemith: Kammermusik No. 1 With Finale 1921,...  
2558  An Italian Songbook - Works Of Bellini, Donize...  
2646                      Boulez: ...Explosante-Fixe...  


In [97]:
mask_both_null = df['artist'].isna() & df['workers'].isna()
subset = df[mask_both_null]

artist_categories = [
    'Best New Artist', 'Best New Artist Of', 'Best Producer Of The Year', 
    'Producer Of The Year', 'Classical Producer Of The Year', 'Remixer Of The Year'
]
# Función para extraer el nombre del artista de 'nominee'
def impute_artist(nominee, category):
    if pd.isna(nominee):
        return None
    # Si la categoría es de artistas o productores, usar 'nominee' directamente
    if any(cat in category for cat in artist_categories):
        return nominee
    # Extraer artista antes de guion en categorías clásicas mixtas
    match = re.match(r'^([^-\(]+?)\s*[-–]\s*.*$', nominee)
    if match:
        return match.group(1).strip()
    # Si parece un título largo, no imputar
    if len(nominee.split()) > 3 or any(kw in nominee.lower() for kw in ['album', 'song', 'works of']):
        return None
    # Fallback: usar 'nominee' si no es claramente un título
    return nominee


# Aplicar la extracción
subset['artist'] = subset.apply(lambda row: impute_artist(row['nominee'], row['category']), axis=1)

C:\Users\Michel Burgos\AppData\Local\Temp\ipykernel_18440\1156722351.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['artist'] = subset.apply(lambda row: impute_artist(row['nominee'], row['category']), axis=1)


In [98]:
df.loc[mask_both_null, 'artist'] = subset['artist']
print("=== Valores nulos después de la imputación ===")
print(df[['artist', 'workers', 'nominee']].isnull().sum())

=== Valores nulos después de la imputación ===
artist     1654
workers    2159
nominee       0
dtype: int64


In [99]:
mask_artist_null_workers_notnull = df['artist'].isna() & df['workers'].notna()
workers_rows = df[mask_artist_null_workers_notnull]


In [100]:
print("\n=== Ejemplos únicos de 'workers' (hasta 10) ===")
unique_workers = workers_rows['workers'].dropna().unique()[:100]
for i, worker in enumerate(unique_workers, 1):
    print(f"{i}. {worker}")


=== Ejemplos únicos de 'workers' (hasta 10) ===
1. Billie Eilish O'Connell & Finneas O'Connell, songwriters (Billie Eilish)
2. Natalie Hemby, Lady Gaga, Hillary Lindsey & Lori McKenna, songwriters (Lady Gaga)
3. Brandi Carlile, Phil Hanseroth, Tim Hanseroth & Tanya Tucker, songwriters (Tanya Tucker)
4. Ruby Amanfu, Sam Ashworth, D. Arcelious Harris, H.E.R. & Rodney Jerkins, songwriters (H.E.R.)
5. Taylor Swift, songwriter (Taylor Swift)
6. Jack Antonoff & Lana Del Rey, songwriters (Lana Del Rey)
7. Tom Barnes, Lewis Capaldi, Pete Kelleher, Benjamin Kohn & Sam Roman, songwriters (Lewis Capaldi)
8. Steven Cheung, Eric Frederic, Melissa Jefferson & Jesse Saint John, songwriters (Lizzo)
9. Gary Clark Jr., songwriter (Gary Clark Jr.)
10. Danny Carey, Justin Chancellor, Adam Jones & Maynard James Keenan, songwriters (Tool)
11. George Daniel, Adam Hann, Matthew Healy & Ross MacDonald, songwriters (The 1975)
12. Ezra Koenig, songwriter (Vampire Weekend)
13. Brittany Howard, songwriter (Britta

In [101]:
# Función para extraer el artista entre paréntesis
def extract_artist_from_parentheses(workers):
    if pd.isna(workers):
        return None
    # Buscar nombre entre paréntesis al final
    paren_match = re.search(r'\(([^)]+)\)$', workers)
    if paren_match:
        return paren_match.group(1).strip()
    return None  # Dejar como None si no hay paréntesis

# Aplicar imputación
workers_rows['artist'] = workers_rows['workers'].apply(extract_artist_from_parentheses)
df.loc[mask_artist_null_workers_notnull, 'artist'] = workers_rows['artist']

C:\Users\Michel Burgos\AppData\Local\Temp\ipykernel_18440\3342724207.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  workers_rows['artist'] = workers_rows['workers'].apply(extract_artist_from_parentheses)


In [102]:
df.head(30)

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,1
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,1
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,1
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,1
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,1
5,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Old Town Road,Lil Nas X Featuring Billy Ray Cyrus,"Andrew ""VoxGod"" Bolooki, Jocelyn “Jozzy” Donal...",https://www.grammy.com/sites/com/files/styles/...,1
6,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Truth Hurts,Lizzo,"Ricky Reed & Tele, producers; Chris Galland, M...",https://www.grammy.com/sites/com/files/styles/...,1
7,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Sunflower,Post Malone & Swae Lee,"Louis Bell & Carter Lang, producers; Louis Bel...",https://www.grammy.com/sites/com/files/styles/...,1
8,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Album Of The Year,"When We All Fall Asleep, Where Do We Go?",Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,1
9,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Album Of The Year,"I,I",Bon Iver,"Brad Cook, Chris Messina & Justin Vernon, prod...",https://www.grammy.com/sites/com/files/styles/...,1


In [103]:
print("\n=== Valores nulos después de la imputación desde paréntesis ===")
print(df[['artist', 'workers', 'nominee']].isnull().sum())


=== Valores nulos después de la imputación desde paréntesis ===
artist      321
workers    2159
nominee       0
dtype: int64


In [104]:
print("\n=== Ejemplos únicos de 'workers' (hasta 10) ===")
mask_artist_null_workers_notnull = df['artist'].isna() & df['workers'].notna()
workers_rows = df[mask_artist_null_workers_notnull]
unique_workers = workers_rows['workers'].dropna().unique()[:327]
for i, worker in enumerate(unique_workers, 1):
    print(f"{i}. {worker}")


=== Ejemplos únicos de 'workers' (hasta 10) ===
1. Randy Brecker, soloist
2. Melissa Aldana, soloist
3. Julian Lage, soloist
4. Branford Marsalis, soloist
5. Christian McBride, soloist
6. Kirk Franklin; Kirk Franklin, songwriter
7. Gloria Gaynor Featuring Yolanda Adams; Bryan Fowler, Gloria Gaynor & Chris Stevens, songwriters
8. Travis Greene Featuring Jekalyn Carr
9. Koryn Hawthorne Featuring Natalie Grant
10. Tasha Cobbs Leonard; Tony Brown, Brandon Lake, Tasha Cobbs Leonard & Nate Moore, songwriters
11. for KING & COUNTRY & Dolly Parton; Josh Kerr, Jordan Reynolds, Joel Smallbone, Luke Smallbone & Tedd Tjornhom, songwriters
12. Casting Crowns; Mark Hall, Bernie Herms & Matthew West, songwriters
13. Danny Gokey; Danny Gokey, Ethan Hulse & Colby Wedgeworth, songwriters
14. Tauren Wells
15. Zach Williams; Ethan Hulse, Andrew Ripp, Jonathan Smith & Zach Williams, songwriters
16. Hildur Guðnadóttir, composer
17. Alan Silvestri, composer
18. Ramin Djawadi, composer
19. Hans Zimmer, compo

In [105]:

def extraer_artista(worker):
    if pd.isnull(worker):
        return None

    # Regla 1: Buscar patrones como "X, soloist" o "X, composer"
    m = re.match(r"([^,;]+), (soloist|composer|conductor|artist)", worker)
    if m:
        return m.group(1).strip()

    # Regla 2: Capturar antes del primer punto y coma si hay Featuring o múltiples artistas
    m = re.match(r"(.+?(Featuring|&| and ).*?)(;|,|$)", worker, re.IGNORECASE)
    if m:
        return m.group(1).strip()

    return worker.strip()

# Aplicar a un DataFrame
df['artist'] = df['artist'].fillna(df['workers'].apply(extraer_artista))


In [106]:
df.isna().sum()

year               0
title              0
published_at       0
updated_at         0
category           0
nominee            0
artist             0
workers         2159
img             1336
winner             0
dtype: int64

In [108]:
df["artist"].value_counts()

artist
(Various Artists)                 66
Various Artists                   44
Chicago Symphony Orchestra        29
John Williams                     24
U2                                22
                                  ..
Jonas Brothers                     1
Shawn Mendes & Camila Cabello      1
Elvis Costello & The Imposters     1
Lewis Capaldi                      1
Black Pumas                        1
Name: count, Length: 2465, dtype: int64

In [109]:
df["artist"] = df["artist"].replace({"(Various Artists)": "Various Artists"})

In [110]:
df["artist"].value_counts()

artist
Various Artists                                           110
Chicago Symphony Orchestra                                 29
John Williams                                              24
Henry Mancini                                              22
U2                                                         22
                                                         ... 
Skrillex And Diplo                                          1
Snarky Puppy & Metropole Orkest                             1
Ghost                                                       1
Kendrick Lamar Featuring Bilal, Anna Wise & Thundercat      1
Bonobo                                                      1
Name: count, Length: 2464, dtype: int64

In [111]:
df.head()

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,1
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,1
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,1
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,1
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,1


In [117]:
df = df.drop(columns= ['published_at', 'updated_at', 'img'])

In [118]:
df.rename(columns= {'winner': 'is_nominated'}, inplace = True)

In [119]:
df.head()

,year,title,category,nominee,artist,workers,is_nominated
0,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",1
1,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",1
2,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",1
3,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",1
4,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",1


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4779 entries, 0 to 4809
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          4779 non-null   int64 
 1   title         4779 non-null   object
 2   category      4779 non-null   object
 3   nominee       4779 non-null   object
 4   artist        4779 non-null   object
 5   workers       2620 non-null   object
 6   is_nominated  4779 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 298.7+ KB
